In [0]:
spark.conf.set(
    "fs.azure.account.key.stetlprojectdeveus.dfs.core.windows.net",
    dbutils.secrets.get(scope="kv-etl-scope", key="storage-account-key")
)


In [0]:
raw_csv_path = "abfss://datalake@stetlprojectdeveus.dfs.core.windows.net/raw/Extended_Employee_Performance_and_Productivity_Data.csv"

In [0]:
bronze_output_path = "abfss://datalake@stetlprojectdeveus.dfs.core.windows.net/bronze/"


## 2. Read the CSV File 

In [0]:
df_bronze = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(raw_csv_path)


In [0]:
# Display the first few rows
display(df_bronze)

# Print the schema to understand the inferred column types
df_bronze.printSchema()


In [0]:
row_count = df_bronze.count()
print(f"Number of rows in raw CSV: {row_count}")


### Perform Minimal Validation


In [0]:
from pyspark.sql.functions import col

df_bronze_clean = df_bronze.filter(col("Employee_ID").isNotNull())


In [0]:
# Check the number of rows after filtering
print(f"Row count before: {df_bronze.count()}")
print(f"Row count after : {df_bronze_clean.count()}")

# Optionally display some records
display(df_bronze_clean)


In [0]:
df_bronze_clean.write.format("delta") \
    .mode("overwrite") \
    .save(bronze_output_path)
